In [91]:
import pandas as pd
import math
import nltk

In [92]:
data = pd.read_csv('SentiLex-lem-PT02.txt',header=None)
df = data[0].str.split(';',n=1,expand=True)
df = df[0].str.split('.',n=1,expand=True)
data['expressao'] = df[0]
data['PoS'] = df[1]

df = data[0].str.split(':',n=1,expand=True)
df = df[0].str.split(';',n=1,expand=True)

data['TG'] = df[1]

df = data[0].str.split(';',n=2,expand=True)
df = df[2].str.split(';ANOT',n=2,expand=True)
data['POL'] = df[0]

df = data[0].str.split(';',n=3,expand=True)
data['ANOT'] = df[3]

df = data['POL'].str.split(';',n=1,expand=True)
data['POL'] = df[0]
data['POL2'] = df[1]


In [93]:
# formata dataframe
data['PoS'] = data['PoS'].str.replace('PoS=','')
data['TG'] = data['TG'].str.replace('TG=','')
data['POL'] = data.POL.replace(regex=['.*='],value='')
data['POL2'] = data.POL2.replace(regex=['.*='],value='')
data['ANOT'] = data.ANOT.replace(regex=['P.*;'],value='')
data['ANOT'] = data['ANOT'].str.replace('ANOT=','')
data = data.drop(columns=[0])

df = data['ANOT'].str.split(';',n=1,expand=True)
data['ANOT'] = df[0]
data['REV'] = df[1]
data['REV'] = data['REV'].str.replace('REV=','')
senti_data = data
senti_data.head(30)


,expressao,PoS,TG,POL,ANOT,POL2,REV
0,à-vontade,N,HUM,1,MAN,None,None
1,abafado,Adj,HUM,-1,JALC,None,None
2,abafante,Adj,HUM,-1,MAN,None,None
3,abaixado,Adj,HUM,-1,JALC,None,None
4,abalado,Adj,HUM,-1,JALC,None,None
5,abalizado,Adj,HUM,1,JALC,None,None
6,abalroado,Adj,HUM,-1,MAN,None,None
7,abalroar,V,HUM,1,MAN,-1,None
8,abanar,V,HUM,1,MAN,-1,None
9,abandalhado,Adj,HUM,-1,MAN,None,None


In [105]:
tweet_data = pd.read_csv('final_set.csv',usecols=['text'])
tweet_data

,text
0,#b17 neles
1,deus proteja capitao #bolsonaro17
2,#elesim god help us
3,voto @jairbolsonaro #elesim #mbl #ordemeprog...
4,parece eleicao domingo vai ser igual crianca b...
...,...
138,#haddadnaoecristao latim : clube bilderberg
139,fara passeio curitiba jean willis ta camara f...
140,olsominions usando tag tipo defendem cara apoi...
141,#haddadnaoecristao hora fazer foto fake respe...


In [97]:
def get_polarity(tokens):
    print(senti_data)

In [106]:
nltk.word_tokenize(tweet_data.iloc[0][0])
# get_polarity()

['#', 'b17', 'neles']